# Practical MCMC 

Vediamo un po' di pacchetti che le persone utilizzano e impariamo il mestiere!

Ho fatto MCMC run. How do I diagnose things are going well? 

1. Check Acceptance Rate: not too high, not too low. Per gli algoritmi con Metropolis-Hastings, from 20% to 50%
2. Check Traceplots: controlla che i parametri sono "mixed well"
3. Check Autocorrelation Length: se una Markov-Chain è dipendente in ogni suo step dal punto precedente, e tu vuoi punti indipendenti tra loro per fare sampling di p.d.f.? => THINNING of the chain: BUTTI un po' di punti in mezzo ( autocorrelation length: quanto è lunga la "memoria" della catena? In teoria è 1, in pratica è un po' più lunga! ) tra i punti, e quelli rimasti "non" hanno legami con quelli prima. **Se vuoi interpretare i numeri che stai tirando fuori come una p.d.f., devi fare thinning della catena!**

##### Optimizing Sampling

- **AM: Adaptive Metropolis** <br> Key Parameters in MH è il "quanto vuoi saltare". In uno schema adaptive, uso la covariance matrix per "decidere" il salto. Faccio derivate della $L$, tiro fuori la matrice di covarianza nel punto, e a seconda di quanto sono su un picco accorcio il salto ( o lo allungo sul "piatto" ). <br> Il thinning della catena risolve il problema citato nel notebook. Non è più MCMC, ma ok.

- **SCAM: Single Component Adaptive Metropolis** <br> What direction should I jump in parameter space? Vedi disegno sulla lavagna, "pizza" di posterior in diagonale. Praticamente "ruoti" il sistema in modo tale da muoverti "bene" e in modo efficiente lungo la posterior!

- **DE: Differential Evolution** <br> Non importa tanto. 

- **HMC: Hamiltonian Monte Carlo** <br> Estremamente efficiente, estremamente veloce. All'atto pratico è difficile da usare, perché servono le derivate della $L$ e non solo la $L$! Se sai la $L$ in modo analitico, molto bene. Però se usi i metodi numerici, soprattutto per una $L$ che fa le bizze con le derivate, auguri! Formulazione molto elegante, molto utile, ma molto ristretta ad un piccolo numero di problemi.

Any model package che useremo makes use of a blend of these techniques. Il software moderno implementa i vari strumenti di diagnostica per "capire" come sta andando il mapping della posterior, e "swticha" tra i vari metodi per andare avanti a mappare.

Ok, abbiamo visto varie "versioni" del Metropolis-Hastings.

---

### MCMC Parameter Estimation In The Wild

Vediamo due pacchetti potenti e molto popolari: `emcee` and `PyMC`. `PTMCMCSampler` è meno usato, ma è popolare in GW.

##### emcee

> The emcee package (*also known as MCMC Hammer, which is in the running for best Python package name in history*) is a Pure Python package written by Astronomer Dan Foreman-Mackey. It is a lightweight package which implements a fairly sophisticated Affine-invariant Hamiltonian MCMC. Because the package is pure Python (i.e. it contains no compiled extensions) it is extremely easy to install; with pip, simply type at the command-line "pip install emcee".

Emcee does not have much specific boilerplate code; it simply requires you to pass it a Python function which returns a value proportional to the log-posterior probability, and returns samples from that posterior.*

##### PyMC

> The PyMC package has many more features than emcee, including built-in support for efficient sampling of common prior distributions. PyMC by default uses the classic Metropolis-Hastings sampler, one of the earliest MCMC algorithms. For performance, it uses compiled fortran libraries, so it is less trivial to install using tools like pip. PyMC binaries for many systems can be quite easily installed with conda.*

More details about PyMC are available from [the pyMC User Guide](https://pymc-devs.github.io/pymc/), but note that we are going to be using [PyMC3](https://docs.pymc.io/).

Questi due pacchetti mappano MULTIPLE CHAINS AT THE SAME TIME, e poi le "pesano". In pratica, puoi fare andare multipli MCMC in parallelo! #TantaRoba

### PTMCMCSampler

> This is a bare-bones sampler, requiring only that the user provide a log-likelihood function and a log-prior function. The user can change the relative amounts of AM, SCAM, and DE being used to propose new points to jump to. The great thing about this sampler is that one can add new custom proposal schemes according to the user's preference. 

> Also, this sampler implements **parallel tempering MCMC (PTMCMC)**. We won't go into detail about this, but suffice it to say that multiple copies of the MCMC are run in parallel, where the copies actually sample from different roots of the likelihood. The higher roots we take, the more the likelihood is flattened out, making it easier to sample. These "rooted" chains then communicate back to the main chain, improving exploration of the parameter space. All these chains can actually be post-processed to get the Bayesian evidence too, in a scheme called **thermodynamic integration**.

Esempio: supponiamo che la $L$ sia come la fa vedere nella lavagna. La "smooth-i" con un KDE? ok. Ne fai un'altra con una bandwidth più fine? Ok. Uno trova le tails, l'altro trova i dettagli. Le due chain "si scambiano informazioni" in qualche modo, ed è questo il trucco magico del parallel tempering. E' UN MCMC SCHEME CHE E' MOLTO UTILE PER TROVARE L'EVIDENCE.

---

Esempio sul suo notebook.

---

### pymc3

PyMC3 è la versione Python-3 di PyMC. Vedi il suo notebook, a me dà errore.

Alcuni algoritmi fanno una specie di esplorazione preliminare, poi "congelano" i parametri e fanno il MCMC "vero e proprio". ( Scelgo come fare il salto, e poi faccio MCMC con quel salto. ) Resta da decidere "quanto è lungo il burning period".

---

### Gibbs Sampling

**Extremely useful ma solo per casi specifici.** Problema di MCMC: è rallentato dal fatto che deve scegliere e soprattutto RIFIUTARE i punti. Il Gibbs sampling NON li rifiuta, ma devi "esprimere il problema" in un modo estremamente specifico.

Supponiamo che sappiamo come fare *sample* da probability distribution. Se il problema è tale per cui fare sampling dalla conditional probability è "fattibile", allora fai GIBBS SAMPLING.

To break this algorithm up into words, we

- We initialize the sampler at some random point in parameter space.


- Fix all parameters except the first one. Draw a random value from the conditional posterior probability distribution of this first parameter **given** the fixed values of all other parameters.


- Fix all parameters except the second one. Draw a random value from the conditional posterior probability distribution of this second parameter **given** the fixed values of all other parameters **and the updated value of the first parameter**.


- *et cetera* for all parameters.


- Repeat for many Gibbs steps.

PRO: Si può dimostrare che non devi rifiutare, virtually, any points. I draws sono una FAIR representation of the posterior, con una autocorrelation length di 1 (?)

CONTRO: ti serve una conditional probability, e devi saperne la forma funzionale! That's where **conjugate priors** come in handy.

Esempio sul suo notebook. ESTREMAMENTE EFFICIENTE, **BUT** you need to sample the conditionals! Supponiamo che riesci a farlo per 12/15 parametri: splitti il problema, almeno "ti semplifichi la vita" un po' e fai "nesting" di MH in Gibbs Sampling.

---

# TIME TO GET YOUR HANDS DIRTY

Time domain astronomy: monitoring the same patch of the sky over time is interesting! 


---